In [1]:
import numpy as np
np.set_printoptions(suppress=True, precision=2)

from dm_control import mujoco
import matplotlib.pylab as plt

import time

import pinocchio as pin

import robot_properties_solo
from robot_properties_solo.config import Solo12Config

from mujoco_utils import MujocoWrapper, MujocoApplication

In [2]:
# Load the model of Solo12 and create a mujoco simulation.
physics = mujoco.Physics.from_xml_path('world_solo12.xml')

# Load Solo12 as a pinocchio robot.
pin_robot = Solo12Config.buildRobotWrapper()

# Create wrapper to make interaction between Mujoco <-> Pinocchio easy.
jnt_names = [
    'base',
    'FL_HAA',
    'FL_HFE',
    'FL_KFE',
    'FR_HAA',
    'FR_HFE',
    'FR_KFE',
    'HL_HAA',
    'HL_HFE',
    'HL_KFE',
    'HR_HAA',
    'HR_HFE',
    'HR_KFE'
]
robot = MujocoWrapper(physics, pin_robot, jnt_names, [5, 9, 13, 17])

In [3]:
app = MujocoApplication(physics, frame_rate=12)

In [4]:
trot = np.load('trot_traj.npz')

xs_traj = trot['xs_plan']
Fs_traj = trot['f_plan']
N = xs_traj.shape[0]

q_traj = xs_traj[:, :19]
v_traj = xs_traj[:, 19:]

com_pos_traj = np.zeros((q_traj.shape[0], 3))
com_vel_traj = np.zeros((q_traj.shape[0], 3))

eff_pos_traj = np.zeros((q_traj.shape[0], 4, 3))
eff_vel_traj = np.zeros((q_traj.shape[0], 4, 3))

eff_ids = Solo12Config.end_eff_ids

# Compute the endeffector position and velocity.
for ti, (q, v) in enumerate(zip(q_traj, v_traj)):
    pin_robot.computeJointJacobians(q)
    pin_robot.framesForwardKinematics(q)
    com_pos_traj[ti], com_vel_traj[ti] = pin_robot.com(q, v)
    
    for i, eff_id in enumerate(eff_ids):
        J = pin_robot.getFrameJacobian(eff_id, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)
        
        eff_pos_traj[ti, i] = pin_robot.data.oMf[eff_id].translation
        eff_vel_traj[ti, i] = (J @ v)[:3]


In [5]:
from mim_control_cpp import (
    ImpedanceController,
    CentroidalPDController,
    CentroidalForceQPController,
)

robot_config = Solo12Config
robot.nb_ee = 4 # HACK
mu = 0.4
kc = [200, 200, 200]
dc = [100, 100, 100]
kb = [100, 100, 200]
db = [50.0, 50.0, 200.0]
qp_penalty_weights = [5e5, 5e5, 5e5, 1e6, 1e6, 1e6]

# Create impedance controllers
root_name = "universe"
endeff_names = Solo12Config.end_effector_names
end_eff_ids = [
    pin_robot.model.getFrameId(ee_name) for ee_name in endeff_names
]
imp_ctrls = [ImpedanceController() for _ in endeff_names]
for i, c in enumerate(imp_ctrls):
    c.initialize(pin_robot.model, root_name, endeff_names[i])

# create centroidal controller
q_init = np.zeros(pin_robot.nq)
q_init[7] = 1
centrl_pd_ctrl = CentroidalPDController()
centrl_pd_ctrl.initialize(
    2.5, np.diag(pin_robot.mass(q_init)[3:6, 3:6])
)
force_qp = CentroidalForceQPController()
force_qp.initialize(robot.nb_ee, mu, qp_penalty_weights)

# Reset the robot to some initial state.
q0 = robot_config.initial_configuration
q0[0] = 0.0
dq0 = robot_config.initial_velocity
robot.reset_state(q0, dq0)

# Desired center of mass position and velocity.
x_com = [0.0, 0.0, 0.25]
xd_com = [0.0, 0.0, 0.0]
# The base should be flat.
x_ori = [0.0, 0.0, 0.0, 1.0]
x_angvel = [0.0, 0.0, 0.0]
# Alle end-effectors are in contact.
cnt_array = robot.nb_ee * [1]

# impedance gains
kp = np.array([200, 200, 200, 0, 0, 0])
kd = np.array([10.0, 10.0, 10.0, 0, 0, 0])

# Desired leg length
x_des = [
    0.195,
    0.147,
    0.015,
    0.195,
    -0.147,
    0.015,
    -0.195,
    0.147,
    0.015,
    -0.195,
    -0.147,
    0.015,
]
xd_des = pin.Motion(np.zeros(3), np.zeros(3))

In [6]:
# Replay the trajectory in the mujoco simulator
# by resetting the state continously.
N = 250

for ti in range(N):
    if ti % 10 == 0:
        robot.reset_state(q_traj[ti], v_traj[ti])
        app.redraw()
        time.sleep(0.1)

In [7]:
start = time.time()

robot.reset_state(q_traj[0], v_traj[0])

N = 1000
for ti in range(N):
    # Read the final state and forces after the stepping.
    q, dq = robot.get_state()

    quat = pin.Quaternion(q[6], q[3], q[4], q[5])
    quat.normalize()

    com_pos, com_vel = pin_robot.com(q, dq)
    
    # computing forces to be applied in the centroidal space
    centrl_pd_ctrl.run(
        kc,
        dc,
        kb,
        db,
#         q[:3],
        com_pos,
        com_pos_traj[ti],
#         quat.toRotationMatrix().dot(dq[:3]), # local to world frame
        com_vel,
        com_vel_traj[ti],
        q[3:7],
        q_traj[ti, 3:7],
        dq[3:6],
        v_traj[ti, 3:6],
    )

    w_com = np.zeros(6)
    w_com[2] += 9.81 * 2.5
    w_com += centrl_pd_ctrl.get_wrench()

    # distributing forces to the active end effectors
    pin_robot.framesForwardKinematics(q)
    com = pin_robot.com(q)
    rel_eff = np.array(
        [pin_robot.data.oMf[i].translation - com for i in end_eff_ids]
    ).reshape(-1)
  
    cnt_array = np.ones(4)

    for i in range(20):
        t_end = 250 * (i + 1)
        t_start = t_end - 200
        
        if ti >= t_start and ti <= t_end:
            if i % 2 == 0:
                cnt_array = [1, 0, 0, 1]
            else:
                cnt_array = [0, 1, 1, 0]
                
            break
                
        if ti < t_start:
            break
    
    force_qp.run(w_com, rel_eff, cnt_array)
    ee_forces = force_qp.get_forces()
    
#     ee_forces += F_traj[ti]

    # passing forces to the impedance controller
    tau = np.zeros(robot.nb_dof)
    for i, imp_ctrl in enumerate(imp_ctrls):
        imp_ctrl.run(
            q,
            dq,
            np.array([300, 300, 300, 0, 0, 0]),
            np.array([10.0, 10.0, 10.0, 0, 0, 0]),
            1.0,
            pin.SE3(np.eye(3), eff_pos_traj[ti, i]),
            pin.Motion(eff_vel_traj[ti, i], np.zeros(3)),
            pin.Force(
                np.array(ee_forces[3 * i : 3 * (i + 1)]), np.zeros(3)
            ),
        )
        tau += imp_ctrl.get_joint_torques()

    # passing torques to the robot
    robot.send_joint_command(tau)
    
    # Uses a sleep factor of 1x:
    # Means the robot runs 1x as fast as realtime assuming the rendering time
    # and running the controller allows it in the time budget.
    app.step(sleep=0.1)

app.redraw()
    
print(time.time() - start)

9.389265775680542


In [283]:
app.close()